In [2]:
import os
import tempfile
import time
from ansys.aedt.core import Hfss

AEDT_VERSION = "2024.2"
NUM_CORES = 4
NG_MODE = False

temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")

project_name = os.path.join(temp_folder.name, "Spherical_Array.aedt")
design_name = "Vertical_Array"
hfss = Hfss(version=AEDT_VERSION,
            non_graphical=NG_MODE,
            project=project_name,
            design = design_name,
            new_desktop=True,
            solution_type="Modal",
            )

PyAEDT INFO: Python version 3.11.6 (tags/v3.11.6:8b6ee5b, Oct  2 2023, 14:57:12) [MSC v.1935 64 bit (AMD64)].
PyAEDT INFO: PyAEDT version 0.16.1.
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\ITA\AppData\Local\Temp\pyaedt_ITA_63aa3973-dff3-435e-9ddb-998c2548824a.log is enabled.
PyAEDT INFO: Log on AEDT is enabled.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT with gRPC plugin.
PyAEDT INFO: New AEDT session is starting on gRPC port 50347.
PyAEDT INFO: Electronics Desktop started on gRPC port: 50347 after 15.287916898727417 seconds.
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\v242\Win64
PyAEDT INFO: Ansoft.ElectronicsDesktop.2024.2 version started with process ID 4216.
PyAEDT INFO: Project Spherical_Array has been created.
PyAEDT INFO: Added design 'Vertical_Array' of type HFSS.
PyAEDT INFO: Aedt Objects correctly read


In [3]:
# Variables
hfss["a"] = "100mm"
a = 0.1
hfss["numseg"] = "18"
hfss["h"] = "1.52mm"
h=0.00152
hfss["Dthetaa"] = "32.7277*1.0226deg"
hfss["Dphia"] = "32.4585*1.0226deg"
hfss["phip1"] = "94.2913deg"
hfss["thetap1"] = "180deg-95.8012deg"
hfss["Rteflon"] = "2.05mm"
Rteflon= 0.00205
hfss["rprobe"] = "0.65mm"
rprobe = 0.00065
hfss["Hprobe"] = "15.07mm"
Hprobe = 0.01507
orange = [255,128,64]
red = [255,0,0]
blue = [0,255,255]

In [4]:
from pyaedt.modeler.cad.polylines import PolylineSegment

try:
    for i in range(4):
        hfss.modeler.coordinate_systems[0].delete()
    hfss.modeler.set_working_coordinate_system("Global")
except:
    pass
# Generate Geometry
start_point = ["0mm", "0mm", "a"]
center_point = ["0mm", "0mm", "0mm"]

Gnd_Sphere_Name = "Gnd_Sphere"
Gnd_Sphere = hfss.modeler.create_polyline(
    points=[start_point, start_point],
    segment_type=[PolylineSegment(
        segment_type="AngularArc",
        arc_center=center_point,
        arc_angle="-180deg",
        arc_plane="ZX",
        num_seg="numseg"
    ), PolylineSegment(
        segment_type="Line"
    )],
    name=Gnd_Sphere_Name
)

hfss.modeler.cover_lines(Gnd_Sphere_Name)

hfss.modeler.sweep_around_axis(Gnd_Sphere_Name, "360deg", number_of_segments="2*numseg")

Gnd_Sphere.material_name = "copper"
Gnd_Sphere.color = orange

start_point = ["0mm", "0mm", "a+h"]

Diel_Sphere_Name = "Diel_Sphere"
Diel_Sphere = hfss.modeler.create_polyline(
    points=[start_point, start_point],
    segment_type=[PolylineSegment(
        segment_type="AngularArc",
        arc_center=center_point,
        arc_angle="-180deg",
        arc_plane="ZX",
        num_seg="numseg"
    ), PolylineSegment(
        segment_type="Line"
    )],
    name=Diel_Sphere_Name
)

hfss.modeler.cover_lines(Diel_Sphere_Name)

hfss.modeler.sweep_around_axis(Diel_Sphere_Name, "360deg", number_of_segments="2*numseg")

Diel_Sphere.material_name = "Arlon AD255A (tm)"

hfss.modeler.subtract(Diel_Sphere_Name, Gnd_Sphere_Name, keep_originals=True)

hfss.modeler.fit_all()

Probe_Sphere_Name = "Patch_Sphere"
Probe_Sphere = hfss.modeler.create_polyline(
    points=[["0mm", "0mm", "a+3*h"], ["0mm", "0mm", "a+3*h"]],
    segment_type=[PolylineSegment(
        segment_type="AngularArc",
        arc_center=center_point,
        arc_angle="-180deg",
        arc_plane="ZX",
        num_seg="numseg"
    ), PolylineSegment(
        segment_type="Line"
    )],
    name=Probe_Sphere_Name
)

hfss.modeler.cover_lines(Probe_Sphere_Name)

hfss.modeler.sweep_around_axis(Probe_Sphere_Name, "360deg", number_of_segments="2*numseg")

hfss.modeler.subtract(Probe_Sphere_Name, [Gnd_Sphere_Name, Diel_Sphere_Name], keep_originals=True)

## Ciclo for
for i in range(1,5):
    Patch_Name = "Patch" + str(i)
    Patch = hfss.modeler.create_polyline(
        points=start_point,
        segment_type=PolylineSegment(
            segment_type="AngularArc",
            arc_center=center_point,
            arc_angle="-180deg",
            arc_plane="ZX",
            num_seg="numseg"
        ),
        name=Patch_Name
    )

    hfss.modeler.sweep_around_axis(Patch_Name, "360deg", number_of_segments="2*numseg")

    hfss["Alpha" + str(i)] = "0deg"
    hfss["Beta" + str(i)] = str(-100+40*i)+"deg"
    hfss["Gamma" + str(i)] = "0deg"

    coord = hfss.modeler.create_coordinate_system(mode = "zxz", origin=center_point, reference_cs="Global",name=Patch_Name+"_CS", phi = "Alpha" + str(i), theta = "Beta"+str(i))

    Cylinder_Name = "Cylinder"+str(i)
    Cylinder = hfss.modeler.create_polyline(
        points=[["0mm","0mm","a+2*h"],["0mm","a+2*h","a+2*h"],["0mm","a+2*h","-a-2*h"],["0mm","0mm","-a-2*h"],["0mm","0mm","a+2*h"]],
        segment_type=[PolylineSegment(
            segment_type="Line",
        ), PolylineSegment(
            segment_type="Line"
        ), PolylineSegment(
            segment_type="Line"
        ), PolylineSegment(
            segment_type="Line"
        )],
        name=Cylinder_Name
    )
    hfss.modeler.cover_lines(Cylinder_Name)
    hfss.modeler.rotate(Cylinder, "Z", angle="-Dphia/2")
    hfss.modeler.sweep_around_axis(Cylinder_Name, axis="Z", sweep_angle="Dphia-360deg")
    Cylinder.rotate("Y", "Gamma"+str(i))

    Cone_Upper_Name = "Cone_Up_" + str(i)
    Cone_Upper = hfss.modeler.create_cone(orientation = "Z", origin = ["0mm","0mm","a+2*h"],bottom_radius = "(a+2*h)*tan(90deg-Dthetaa/2)", top_radius=0, height = "-(a+2*h)", name = Cone_Upper_Name)
    Cone_Upper.rotate("Y", "Gamma"+str(i))

    Cone_Lower_Name = "Cone_Down_" + str(i)
    Cone_Lower = hfss.modeler.create_cone(orientation = "Z", origin = ["0mm","0mm","-(a+2*h)"],bottom_radius = "(a+2*h)*tan(90deg-Dthetaa/2)", top_radius=0, height = "a+2*h", name = Cone_Lower_Name)
    Cone_Lower.rotate("Y", "Gamma"+str(i))

    hfss.modeler.subtract(Patch_Name, [Cylinder_Name, Cone_Lower_Name, Cone_Upper_Name], keep_originals=False)
    hfss.assign_finite_conductivity(Patch_Name)

    Probe_Name = "Probe"+str(i)
    Probe = hfss.modeler.create_cylinder(orientation="Z", origin=["0mm","0mm","a+h"], radius="rprobe", height="-Hprobe", name = Probe_Name, material="copper")

    Teflon_Name = "Teflon"+str(i)
    Teflon = hfss.modeler.create_cylinder(orientation="Z", origin=["0mm","0mm","a+h/5"], radius="Rteflon", height="4*h/5-Hprobe", name = Teflon_Name, material="Teflon (tm)")

    hfss.modeler.subtract(Teflon_Name, Probe_Name, keep_originals=True)

    hfss.wave_port(assignment=hfss.modeler[Teflon_Name].faces[0], name = "Wave_Port"+str(i), deembed="Hprobe-h")

    hfss.modeler.rotate([Probe_Name, Teflon_Name], "Y", angle="thetap1")
    hfss.modeler.rotate([Probe_Name, Teflon_Name], "Z", angle="phip1")
    hfss.modeler.rotate([Probe_Name, Teflon_Name], "Y", angle="Gamma"+str(i))
    hfss.modeler.subtract(Gnd_Sphere_Name,[Teflon_Name, Probe_Name], keep_originals=True)
    hfss.modeler.subtract(Teflon_Name, Diel_Sphere_Name, keep_originals=True)
    hfss.modeler.subtract(Diel_Sphere_Name, Probe_Name, keep_originals=True)
    hfss.modeler.subtract(Probe_Name, Probe_Sphere_Name, keep_originals=True)

    ###

    hfss.modeler.set_working_coordinate_system("Global")

    hfss.modeler[Patch_Name].color = orange

hfss.modeler.delete(Probe_Sphere_Name)
for obj in hfss.modeler.object_names:
    hfss.modeler[obj].transparency = 0

PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 1sec
PyAEDT INFO: Materials class has been initialized! Elapsed time: 0m 0sec
PyAEDT INFO: Parsing design objects. This operation can take time
PyAEDT INFO: Refreshing bodies from Object Info
PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec
PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec
PyAEDT INFO: Boundary Finite Conductivity Coating_Patch1 has been created.
PyAEDT INFO: Boundary Wave Port Wave_Port1 has been created.
PyAEDT INFO: Parsing design objects. This operation can take time
PyAEDT INFO: Refreshing bodies from Object Info
PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec
PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec
PyAEDT INFO: Boundary Finite Conductivity Coating_Patch2 has been created.
PyAEDT INFO: Boundary Wave Port Wave_Port2 has been created.
PyAEDT INFO: Parsing design objects. This operation can take time
PyAEDT INFO: Refreshing bodies from Object Info
PyAE

In [5]:
freq_range = [1.45e9, 1.75e9]      # Frequency range for analysis and post-processing.
center_freq = 1575.42e6            # Center frequency
npoints = 31

hfss.create_open_region(frequency=str(center_freq/1e9)+"GHz", boundary = "PML")

setup = hfss.create_setup(name="GPS_Band", Frequency = str(center_freq/1e9)+"GHz", MaximumPasses=20, MinimumPasses=2,MinimumConvergedPasses=5,PercentRefinement=30)

disc_sweep = setup.add_sweep(name="GPS_Discrete", sweep_type="Discrete",
                             RangeStart=str(freq_range[0]/1e9)+"GHz", RangeEnd=str(freq_range[1]/1e9)+"GHz", RangeCount = npoints,
                             SaveFields=True)

PyAEDT INFO: Open Region correctly created.
PyAEDT INFO: Project Spherical_Array Saved correctly
PyAEDT INFO: Parsing C:/Users/ITA/AppData/Local/Temp/tmpi705n6ef.ansys/Spherical_Array.aedt.
PyAEDT INFO: File C:/Users/ITA/AppData/Local/Temp/tmpi705n6ef.ansys/Spherical_Array.aedt correctly loaded. Elapsed time: 0m 5sec
PyAEDT INFO: aedt file load time 4.701508522033691


In [6]:
Rad_3D = hfss.field_setups[0]
print(Rad_3D.name)
Rad_3D.theta_step = "1deg"
Rad_3D.phi_step = "1deg"

3D


In [7]:
variations = hfss.available_variations.nominal_values
variations["Freq"] = [center_freq]
variations["Theta"] = ["All"]
variations["Phi"] = ["All"]
axialratio_ffd_plot = hfss.post.create_report(expressions="dB(AxialRatioValue)",
                                             setup_sweep_name=hfss.nominal_adaptive,
                                             variations=variations,
                                             primary_sweep_variable="Theta",
                                             context="3D",           # Far-field setup is pre-defined.
                                             report_category="Far Fields",
                                             plot_type="Rectangular Plot",
                                             plot_name="Axial Ratio (dB)"
                                            )

PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec
PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec
PyAEDT INFO: Post class has been initialized! Elapsed time: 0m 0sec


In [ ]:
setup.update()
hfss.save_project()
#setup.enable_expression_cache(expressions="S(Wave_Port1,Wave_Port1)", report_type="Modal Solution Data", isrelativeconvergence= False, conv_criteria='0.02')

setup.enable_expression_cache(report_type="Far Fields", expressions="dB(AxialRatioValue)", intrinsics=["Theta='90deg'","Phi='90deg'"], isrelativeconvergence= False, conv_criteria='0.2')

setup.properties

PyAEDT INFO: Project Spherical_Array Saved correctly


{'Name': 'GPS_Band',
 'Enabled': True,
 'Passes': 20,
 'Percent Refinement': 30,
 'Delta S': 0.02,
 'Solution Freq Type': 'Single',
 'Solution Freq Type/Choices': ['Single', 'Multiple', 'Broadband'],
 'Solution Freq': '1.57542GHz',
 'Basis Order': 'First Order',
 'Basis Order/Choices': ['Mixed Order',
  'Zero Order',
  'First Order',
  'Second Order'],
 'Max Refinement': 1000000,
 'Use Max Refinement': False,
 'Use ABC On Ports': False,
 'Solver Type': 'Direct Solver',
 'Solver Type/Choices': ['Direct Solver',
  'Iterative Solver',
  'Domain Solver',
  'Auto Select Direct/Iterative'],
 'Use ABC During Adaptive Passes FEBI': False}

In [ ]:

print(hfss.post.available_report_quantities(report_category="Far Fields", quantities_category="Axial Ratio"))
print(hfss.post.available_quantities_categories(report_category="Far Fields"))
print(hfss.post.available_quantities_categories(report_category="Antenna Parameters"))
print(hfss.post.available_report_solutions())
print(hfss.post.available_report_types)

#print(hfss.post.get_all_report_quantities())

In [ ]:
[
			"NAME:ExpressionCache",
			[
				"NAME:CacheItem",
				"Title:="		, "dB_AxialRatioValue_1",
				"Expression:="		, "dB(AxialRatioValue)",
				"Intrinsics:="		, "Theta=\'90deg\' Phi=\'90deg\' ",
				"IsConvergence:="	, True,
				"UseRelativeConvergence:=", 0,
				"MaxConvergenceDelta:="	, 1,
				"MaxConvergeValue:="	, "0.02",
				"ReportType:="		, "Far Fields",
				[
					"NAME:ExpressionContext",
					"Context:="		, "3D"
				]
			]
		]

In [ ]:
hfss.save_project()
hfss.release_desktop()

In [ ]:
spar_plot = hfss.create_scattering(plot="Return Loss", sweep=interp_sweep.name)
variations = hfss.available_variations.nominal_values
variations["Freq"] = [center_freq]
variations["Theta"] = ["All"]
variations["Phi"] = ["All"]
elevation_ffd_plot = hfss.post.create_report(expressions="db(GainTheta)",
                                             setup_sweep_name=disc_sweep.name,
                                             variations=variations,
                                             primary_sweep_variable="Theta",
                                             context="Elevation",           # Far-field setup is pre-defined.
                                             report_category="Far Fields",
                                             plot_type="Radiation Pattern",
                                             plot_name="Elevation Gain (dB)"
                                            )
elevation_ffd_plot.children["Legend"].properties["Show Trace Name"] = False
elevation_ffd_plot.children["Legend"].properties["Show Solution Name"] = False

report_3d = hfss.post.reports_by_category.far_field("db(RealizedGainTheta)",
                                                      disc_sweep.name,
                                                      sphere_name="3D",
                                                      Freq= [center_freq],)

report_3d.report_type = "3D Polar Plot"
report_3d.create(name="Realized Gain (dB)")

In [34]:
hfss.save_project()
hfss.release_desktop()

# Wait 5 seconds to allow AEDT to shut down before cleaning the temporary directory.
time.sleep(5)

temp_folder.cleanup()

PyAEDT INFO: Project Spherical_Array Saved correctly
PyAEDT INFO: Desktop has been released and closed.
